# Rumour Analysis with Covid Data

In [ ]:
! pip3 install spacy
! python3 -m spacy download en_core_web_sm

## Inference

In [5]:
import dataloader

In [6]:
covid_df = dataloader.load_data(data_file = '../data/covid.data.jsonl', label_file = None, perform_stemming = False)

In [7]:
covid_df

,id,text,retweet_count,favorite_count,question_mark,contains_url,number_urls,contains_media,statuses_count,listed_count,followers_count,friends_count,contains_profile_background_image,reputation_score_1,reputation_score_2,favourites_count,verified,geo_enabled,has_description,length_description
0,1272262651100434433,"According to the New York Times, Warner Bros. ...",28,280,False,False,0,True,2512,112,26179,18,True,1377.842105,0.999275,537,False,False,True,128
1,1287153210990395392,Hurricane Hanna has made landfall in Texas.\n\...,545,658,False,True,1,False,94186,5053,248090,59111,True,4.196948,0.807579,3126,True,True,True,131
2,1266555444283179008,Monkeys on the loose in India with stolen coro...,83,179,False,True,1,True,782525,26725,3948552,266,True,14788.584270,0.999932,10,True,False,True,159
3,1257715199655755779,Eastleigh and Swahili Arabs in Mombasa where c...,304,2306,False,False,0,False,26895,48,176217,1287,True,136.814441,0.992744,3340,False,True,True,149
4,1274338812173393920,"“If Trump felt comfortable having it here, the...",856,2113,False,True,1,False,17452,1834,182389,1229,True,148.283740,0.993301,5846,True,True,True,142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17453,1249502859185590272,I wonder how many lives could’ve been saved if...,2774,11419,False,False,0,False,250958,3871,789505,84254,True,9.370423,0.903572,131219,True,True,True,155
17454,1284050414619459586,The front page on 17th March. The first line r...,259,866,False,False,0,True,27831,1044,98094,914,True,107.206557,0.990758,6092,True,True,True,153
17455,1274505289614725122,Trump just completed the racism trifecta in a ...,408,1007,False,False,0,False,8346,695,69919,88,True,785.606742,0.998729,35,True,True,True,52
17456,1267884642637676545,Here are a few of my photographs from today’s ...,165,366,False,False,0,True,485,35,6131,593,True,10.321549,0.911673,516,False,True,True,50


## Exploratory Data Analysis

Similiar to https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert

In [ ]:
# word_count
covid_df['word_count'] = covid_df['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
covid_df['unique_word_count'] = covid_df['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
covid_df['stop_word_count'] = covid_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
covid_df['url_count'] = covid_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
covid_df['mean_word_length'] = covid_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
covid_df['char_count'] = covid_df['text'].apply(lambda x: len(str(x)))

# punctuation_count
covid_df['punctuation_count'] = covid_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
covid_df['hashtag_count'] = covid_df['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
covid_df['mention_count'] = covid_df['text'].apply(lambda x: len([c for c in str(x) if c == '@']))


In [ ]:
METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length',
                'char_count', 'punctuation_count', 'hashtag_count', 'mention_count']
DISASTER_TWEETS = df_train['target'] == 1

fig, axes = plt.subplots(ncols=2, nrows=len(METAFEATURES), figsize=(20, 50), dpi=100)

for i, feature in enumerate(METAFEATURES):
    sns.distplot(df_train.loc[~DISASTER_TWEETS][feature], label='Not Disaster', ax=axes[i][0], color='green')
    sns.distplot(df_train.loc[DISASTER_TWEETS][feature], label='Disaster', ax=axes[i][0], color='red')

    sns.distplot(df_train[feature], label='Training', ax=axes[i][1])
    sns.distplot(df_test[feature], label='Test', ax=axes[i][1])
    
    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)
        axes[i][j].legend()
    
    axes[i][0].set_title(f'{feature} Target Distribution in Training Set', fontsize=13)
    axes[i][1].set_title(f'{feature} Training & Test Set Distribution', fontsize=13)

plt.show()